# Analysis of Zeus/Gameover HTTP Connections

Dataset collected by Anonymous using Bro. Preprocessed with bash: sed -r 's/^#fields\t//' http.log | sed '/#/d' | tr "\t" "," > http_noG.csv

Dataset further cleaned by removing google searches. It seems unlikely that Zeus/Gameover was making searches. May add back in at a later date. See http_G.csv.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
df = pd.read_csv("http_noG.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ts                 78 non-null     float64
 1   uid                78 non-null     object 
 2   id.orig_h          78 non-null     object 
 3   id.orig_p          78 non-null     int64  
 4   id.resp_h          78 non-null     object 
 5   id.resp_p          78 non-null     int64  
 6   trans_depth        78 non-null     int64  
 7   method             78 non-null     object 
 8   host               78 non-null     object 
 9   uri                78 non-null     object 
 10  referrer           78 non-null     object 
 11  user_agent         78 non-null     object 
 12  request_body_len   78 non-null     int64  
 13  response_body_len  78 non-null     int64  
 14  status_code        78 non-null     int64  
 15  status_msg         78 non-null     object 
 16  info_code          78 non-nu

OK so we can see that the dataset was read properly and everything is of a proper type. Now the question is: What are we looking for? What is the purpose of us going through this dataset?

Just remember to OSEMN. Obtain, Scrub, Explore, Model, iNterpret.
We obtained the data already and processed it into a format we can use. We scrubbed out potentially useless data, though we did save it to a different file just in case. Now we need to know what we have and what it looks like. We also need to do some research on Gameover Zeus and see what behaviors we should look for. 

In [5]:
print(df.head())

             ts                 uid     id.orig_h  id.orig_p       id.resp_h  \
0  1.328383e+09  C2zKxQ1qgZrLXH5Aa6  172.29.0.116       1488  91.191.168.201   
1  1.328383e+09   CA7q4Acu02edsdpw7  172.29.0.116       1489   206.123.104.4   
2  1.328383e+09   CJujChRafzg9JE4Q3  172.29.0.116       1490  209.59.217.102   
3  1.328383e+09   CJujChRafzg9JE4Q3  172.29.0.116       1490  209.59.217.102   
4  1.328383e+09   CJujChRafzg9JE4Q3  172.29.0.116       1490  209.59.217.102   

   id.resp_p  trans_depth method                   host  \
0         80            1    GET    pinardisticaret.com   
1         80            1    GET  talenttechcareers.com   
2         80            1    GET           telecrop.com   
3         80            2    GET           telecrop.com   
4         80            3    GET           telecrop.com   

                                 uri  ... info_msg filename     tags  \
0               /jqCBSmVw/index.html  ...        -        -  (empty)   
1                   

In [43]:

print(df.loc[df['referrer'] != '-'])

              ts                 uid     id.orig_h  id.orig_p       id.resp_h  \
1   1.328383e+09   CA7q4Acu02edsdpw7  172.29.0.116       1489   206.123.104.4   
3   1.328383e+09   CJujChRafzg9JE4Q3  172.29.0.116       1490  209.59.217.102   
4   1.328383e+09   CJujChRafzg9JE4Q3  172.29.0.116       1490  209.59.217.102   
26  1.328383e+09  CPEfxD2ERLvqNKIVnf  172.29.0.116       1503  209.59.217.102   
27  1.328383e+09  C5yXhh1qvtV0whAMP8  172.29.0.116       1520  209.59.217.102   

    id.resp_p  trans_depth method                   host  \
1          80            1    GET  talenttechcareers.com   
3          80            2    GET           telecrop.com   
4          80            3    GET           telecrop.com   
26         80            2    GET           telecrop.com   
27         80            1    GET           telecrop.com   

                         uri  ... info_msg filename     tags  username  \
1            /iwUsQfez/js.js  ...        -        -  (empty)         -   
3   

In [44]:
host_ref = df[['host','referrer']]
print(host_ref[host_ref['referrer']!= '-'])
print()
refs = list(host_ref['referrer'].loc[host_ref['referrer'] != '-'])
for ref in refs :
    print(ref)

                     host                                           referrer
1   talenttechcareers.com     http://pinardisticaret.com/jqCBSmVw/index.html
3            telecrop.com  http://telecrop.com/search.php?page=73a07bcb51...
4            telecrop.com  http://telecrop.com/search.php?page=73a07bcb51...
26           telecrop.com  http://telecrop.com/search.php?page=73a07bcb51...
27           telecrop.com  http://telecrop.com/search.php?page=73a07bcb51...

http://pinardisticaret.com/jqCBSmVw/index.html
http://telecrop.com/search.php?page=73a07bcb51f4be71
http://telecrop.com/search.php?page=73a07bcb51f4be71
http://telecrop.com/search.php?page=73a07bcb51f4be71
http://telecrop.com/search.php?page=73a07bcb51f4be71


The searches in telecrop.com seem interesting. My guess is that the searches are being used to serve up exploit kits.

In [42]:
print(df[['host','uri','resp_mime_types']].loc[host_ref['host'] == 'telecrop.com'])

            host                                uri  \
2   telecrop.com  /search.php?page=73a07bcb51f4be71   
3   telecrop.com           /content/adfp2.php?f=126   
4   telecrop.com                 /content/field.swf   
6   telecrop.com                  /content/rino.jar   
7   telecrop.com                         /com.class   
8   telecrop.com                         /edu.class   
9   telecrop.com                         /net.class   
10  telecrop.com                         /org.class   
11  telecrop.com                   /d.php?f=126&e=0   
18  telecrop.com  /search.php?page=73a07bcb51f4be71   
26  telecrop.com           /content/adfp2.php?f=126   
27  telecrop.com                 /content/field.swf   
28  telecrop.com                   /d.php?f=126&e=4   
29  telecrop.com                  /content/rino.jar   
31  telecrop.com                         /com.class   
33  telecrop.com                         /edu.class   
34  telecrop.com                         /net.class   
35  telecr

So theres a few things that are catching my eye. 
* telecrop.com/search.php?page=73a07bcb51f4be71
* telecrop.com/content/adfp2.php?f=126
* telecrop.com/d.php?f=126&e=4 
* telecrop.com/content/rino.jar 

According to my research rino.jar is a Blackhole java exploit kit. It seems like Gameover Zeus is performing the 'search' with telecrop with a specific md5 which corresponds to the exploit kit. The bot seems like its downloading configuration files with the .class files which come immediately after the rino.jar gets downloaded.

What I also find interesting is the adfp2.php?f=126. They're making a call with a specific parameter which returns an application/pdf mime type. f=126 gets called again with d.php?f=126 but with the additional parameter &e=4 or &e=0 which corresponds to application/x-dosexec mime type. It is likely that the bot is configuring itself.

According to packetsecurity.com d.php?f=126&e=0 refers to Blackhole downloading contacts.exe 

And adfp2.php?f=126 is Blackhole downloading a pdf exploit

According to Michael Sandee of Fox IT Gameover Zeus dynamically configures itself through the victims browser by adding in new fields to the url. This means we'll need to check out the google searches and see if its working with those as well.

In [20]:
df2 = pd.read_csv('http_G.csv')
print(df2.info)

<bound method DataFrame.info of              ts                 uid     id.orig_h  id.orig_p       id.resp_h  \
0  1.328383e+09  C0R8i7194WxLU2yUA2  172.29.0.116       1537  74.125.113.106   
1  1.328383e+09  CtsKmX28RGJhGaII2k  172.29.0.116       1542  74.125.113.106   
2  1.328383e+09  Cz8h1R1A3fQ9FEEXS3  172.29.0.116       1544  74.125.113.106   
3  1.328383e+09  CcAyOQ1ZeNVzsq20S3  172.29.0.116       1546  74.125.113.106   
4  1.328383e+09    CW9aMM9cvwpy0j4Q  172.29.0.116       1547  74.125.113.106   
5  1.328385e+09  CEqUJK2fzgoC2cCozf  172.29.0.116       1613  74.125.113.106   
6  1.328385e+09  CZKpbl4VeTNkPtL3Q9  172.29.0.116       1617   94.63.240.137   
7  1.328385e+09  CHuOHu1LpeWBI3FIc2  172.29.0.116       1621   94.63.240.137   

   id.resp_p  trans_depth method            host  \
0         80            1    GET  www.google.com   
1         80            1    GET  www.google.com   
2         80            1    GET  www.google.com   
3         80            1    GET  www.g

Ok that isn't too much help. Lets keep looking around.

In [28]:
print(df[['host','uri','resp_mime_types']].loc[df['resp_mime_types'] == "application/x-dosexec"])

                   host               uri        resp_mime_types
11         telecrop.com  /d.php?f=126&e=0  application/x-dosexec
13        airplains.com         /melt.exe  application/x-dosexec
14       airportsys.com         /melt.exe  application/x-dosexec
15    eurostats2012.net         /melt.exe  application/x-dosexec
28         telecrop.com  /d.php?f=126&e=4  application/x-dosexec
32        airplains.com         /melt.exe  application/x-dosexec
36         telecrop.com  /d.php?f=126&e=0  application/x-dosexec
37       airportsys.com         /melt.exe  application/x-dosexec
39        airplains.com         /melt.exe  application/x-dosexec
40    eurostats2012.net         /melt.exe  application/x-dosexec
43       airportsys.com         /melt.exe  application/x-dosexec
44    eurostats2012.net         /melt.exe  application/x-dosexec
48  dejangorgievski.com      /pomk/24.exe  application/x-dosexec


Alright so we can see some more strange hosts and suspicious uris. Lets check out some of those hosts.

In [33]:
print(df[['host','uri','referrer']].loc[df['host'] == 'airplains.com'])

             host        uri referrer
13  airplains.com  /melt.exe        -
32  airplains.com  /melt.exe        -
39  airplains.com  /melt.exe        -


In [34]:
print(df[['host','uri','referrer']].loc[df['host'] == 'airportsys.com'])

              host        uri referrer
14  airportsys.com  /melt.exe        -
37  airportsys.com  /melt.exe        -
43  airportsys.com  /melt.exe        -


In [35]:
print(df[['host','uri','referrer']].loc[df['host'] == 'dejangorgievski.com'])

                   host           uri referrer
48  dejangorgievski.com  /pomk/24.exe        -


In [36]:
print(df[['host','uri','referrer']].loc[df['host'] == 'eurostats2012.net'])

                 host        uri referrer
15  eurostats2012.net  /melt.exe        -
40  eurostats2012.net  /melt.exe        -
44  eurostats2012.net  /melt.exe        -


I have a theory. There are urls that are being called before the bot attempts to download melt.exe. Lets check it out.

In [38]:
print(df[['host','uri']].loc[15:44])

                 host                                uri
15  eurostats2012.net                          /melt.exe
16        ewpetro.com                          /melt.exe
17    www.ewpetro.com                          /melt.exe
18       telecrop.com  /search.php?page=73a07bcb51f4be71
19         172.29.0.1             /IGatewayDeviceDescDoc
20         172.29.0.1               /L3ForwardingDescDoc
21         172.29.0.1               /WanCommonIFCDescDoc
22         172.29.0.1         /WanEthernetLinkCfgDescDoc
23         172.29.0.1            /WanIPConnectionDescDoc
24         172.29.0.1                  /WANIPConnCtrlUrl
25         172.29.0.1                  /WANIPConnCtrlUrl
26       telecrop.com           /content/adfp2.php?f=126
27       telecrop.com                 /content/field.swf
28       telecrop.com                   /d.php?f=126&e=4
29       telecrop.com                  /content/rino.jar
30    neoprenolen.com                     /pony/gate.php
31       telecrop.com          

We can see that the bot is nabbing Pony Stealer. It also is trying to grab melt.exe. We're able to see the list of hosts that it has generated through its attempts. Lets see if its successful in grabbing melt.exe .

In [39]:
print(df[['host','uri','response_body_len']].loc[df['uri'] == '/melt.exe'])

                 host        uri  response_body_len
13      airplains.com  /melt.exe             329192
14     airportsys.com  /melt.exe             329192
15  eurostats2012.net  /melt.exe             329192
16        ewpetro.com  /melt.exe                  0
17    www.ewpetro.com  /melt.exe               4437
32      airplains.com  /melt.exe             329192
37     airportsys.com  /melt.exe             329192
39      airplains.com  /melt.exe             329192
40  eurostats2012.net  /melt.exe             329192
41        ewpetro.com  /melt.exe                  0
42    www.ewpetro.com  /melt.exe               4437
43     airportsys.com  /melt.exe             329192
44  eurostats2012.net  /melt.exe             329192
45        ewpetro.com  /melt.exe                  0
46    www.ewpetro.com  /melt.exe               4437


We can see that airplains, airportsys, and eurostats2012 are active. Ewpetro.com and www. ewpetro.com are likely inactive given their small response length. This will help narrow our search down when we investigate. I am unable to determine what melt.exe does. Only that Dark Bay ltd and 'Hacker's Handbook' are related to it. 


## Analysis
Through the Bro logs of Gameover Zeus we are able to detect GoZ's dynamic configuration and get a sense of how the actors are using it. We also know that they are using the Blackhole exploit kit as well as Pony stealer. This will allow us to create rules to detect the actions of the bot much quicker and ideally reduce the potential damage it can cause.